In [6]:
# IMPORTS AND COMMONLY USED FUNCTIONS
# ==> always run first, change server to match the desired one

import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# "global" | "asia" | "jpn" | "kor"
server = "kor"


uid_df = pd.read_json('epic7_user_world_{}.json'.format(server))
users = dict()
for x in range(len(uid_df)):
    u = dict(uid_df.iloc[x]["users"])
    users[u["nick_no"]] = u["nick_nm"]

users2 = dict()
for x in range(len(uid_df)):
    u = dict(uid_df.iloc[x]["users"])
    users2[u["nick_nm"]] = u["nick_no"]
    
heroname_df = pd.read_json('heronames.json')
heronames = dict()
for x in range(len(heroname_df)):
    heronames[heroname_df.iloc[x]["code"]] = heroname_df.iloc[x]["name"]
    
def getHeroName(code):
    return heronames[code]
def getUID(name):
    return users2[name]
def getUsername(uid):
    return users[uid]

In [2]:
# Get all rta data

def generateData(uid, sus_log_p=-30):
    url = "https://epic7.gg.onstove.com/gameApi/getBattleList?nick_no={}&world_code=world_{}&lang=en&season_code=".format(uid, server)
    payload = {}
    response = requests.post(url, json=payload)


    filename = '{userId}.json'.format(userId=uid)
    if response.status_code == 200:
        json_data = response.json()
        with open(filename, 'w') as file:
            json.dump(json_data, file, indent=4)

        battles = pd.read_json(filename)
        battles = dict(battles.loc["battle_list"])['result_body']

        enemies = dict()
        heroes = dict()
        total_picks = 0
        total_battles = 0
        for x in range(len(battles)):
            if battles[x]["season_code"] == "pvp_rta_ss13":
                total_battles = total_battles + 1
                if battles[x]["enemy_nick_no"] in enemies:
                    enemies[battles[x]["enemy_nick_no"]] = enemies[battles[x]["enemy_nick_no"]] + 1
                else: 
                    enemies[battles[x]["enemy_nick_no"]] = 1    
                h = battles[x]["my_deck"]["hero_list"]
                for x2 in range(len(h)):
                    total_picks = total_picks + len(h)
                    if h[x2]["hero_code"] in heroes:
                        heroes[h[x2]["hero_code"]] = heroes[h[x2]["hero_code"]] + 1
                    else:
                        heroes[h[x2]["hero_code"]] = 1

        for hero in heroes.keys():
            heroes[hero] = np.log(heroes[hero]) - np.log(total_picks)

        battle_log_p = [1]*total_battles
        x5 = 0
        picks = [""]*total_battles
        for x in range(len(battles)):
            if battles[x]["season_code"] == "pvp_rta_ss11":   
                h = battles[x]["my_deck"]["hero_list"]
                for x2 in range(len(h)):
                    if x2 == 0:
                        picks[x5] = picks[x5] + getHeroName(h[x2]["hero_code"])
                    else:
                        picks[x5] = picks[x5] + ", " + getHeroName(h[x2]["hero_code"])
                    if h[x2]["hero_code"] in heroes:
                        battle_log_p[x5] = battle_log_p[x5] + heroes[h[x2]["hero_code"]]
                x5 = x5 + 1
        
        print("Generating data for: " + str(uid))
    else:
        print(f'Error: {response.status_code}')
        print(response.text)

In [4]:
rta_players_global = [159055138, 60090739, 63471324, 79680386, 60318328, 139771474,
               66879752, 63467428, 163085629, 188175150, 68183898, 60134130,
               85251058, 206705999, 124552294, 191603279, 177444551, 159620508,
               171276418, 170502668, 61214698, 133232375, 60142509, 85743517,
               191616064, 63908598, 70726715, 155256756, 134489859, 79806553]

rta_players_asia = [71212252, 71559401, 63961422, 77540440, 65920769, 80143521,
                    163752543, 124682864, 137537222, 61210383, 130599173, 
                    159161103, 68673050, 61171745, 65799678]

rta_players_kor = [53509258, 50094501, 57984203, 51245518, 50441743,
                   56413544, 91691789, 53969894, 188829031, 55685318]

rta_players_eu = [182147370]

In [ ]:
# for i in range(len(rta_players_global)):
#     generateData(rta_players_global[i])

In [5]:
# for i in range(len(rta_players_asia )):
#     generateData(rta_players_asia[i])

Generating data for: 71212252
Generating data for: 71559401
Generating data for: 63961422
Generating data for: 77540440
Generating data for: 65920769
Generating data for: 80143521
Generating data for: 163752543
Generating data for: 124682864
Generating data for: 137537222
Generating data for: 61210383
Generating data for: 130599173
Generating data for: 159161103
Generating data for: 68673050
Generating data for: 61171745
Generating data for: 65799678


In [7]:
# for i in range(len(rta_players_kor)):
#     generateData(rta_players_kor[i])

Generating data for: 53509258
Generating data for: 50094501
Generating data for: 57984203
Generating data for: 51245518
Generating data for: 50441743
Generating data for: 56413544
Generating data for: 91691789
Generating data for: 53969894
Generating data for: 188829031
Generating data for: 55685318
